In [1]:
import os
import concurrent
import pandas_datareader.data as web
import pandas as pd
import datetime
from datetime import timedelta
import concurrent.futures
from concurrent.futures import wait, ALL_COMPLETED
import yfinance as yf
import urllib
import urllib.request
import time
from finquant.portfolio import build_portfolio

import pandas_datareader as pdr

from ipywidgets import interactive
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import pycorrelate as pyc
import seaborn as sns
from yahoofinancials import YahooFinancials
#from sympy import symbols, solve

import pandas_market_calendars as mcal
import re
from sklearn.model_selection import train_test_split

from scipy import stats # For in-built method to get PCC
from scipy.ndimage.interpolation import shift

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
#import statsmodels.formula.api as sm
import scipy.stats  as stats
from itertools import cycle
import matplotlib.pyplot as plt

from dask.distributed import Client
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression
from dask.distributed import as_completed

In [2]:
def split_sequences(sequences, n_steps_in, n_steps_out):

    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y)

def crosscorrelation(x, y, maxlag, mode='corr'):
	"""
	Cross correlation with a maximum number of lags.

	`x` and `y` must be one-dimensional numpy arrays with the same length.

	This computes the same result as
		numpy.correlate(x, y, mode='full')[len(a)-maxlag-1:len(a)+maxlag]

	The return vaue has length 2*maxlag + 1.
	"""
	py = np.pad(y.conj(), 2*maxlag, mode='constant')
	T = np.lib.stride_tricks.as_strided(py[2*maxlag:], shape=(2*maxlag+1, len(y) + 2*maxlag),
				   strides=(-py.strides[0], py.strides[0]))
	px = np.pad(x, maxlag, mode='constant')
	if mode == 'dot':       # get lagged dot product
		return T.dot(px)
	elif mode == 'corr':    # gets Pearson correlation
		return (T.dot(px)/px.size - (T.mean(axis=1)*px.mean())) / \
			   (np.std(T, axis=1) * np.std(px))

def unique(list1):

    # intilize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

    return(unique_list)

def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
        
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [3]:
#frequency = output_variable.value
frequency = "Q"

wd = os.getcwd()

w=52*15
#start = datetime.datetime(2010, 1, 1)

#end = datetime.datetime(2013, 1, 27)
end_date = datetime.date.today()

start_date = end_date - timedelta(weeks=w)

pd.set_option('display.max_columns', None) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', None) #replace n with the number of rows you want to see completely

#cores = int(len(os.sched_getaffinity(0)))

print(end_date)
print(start_date)

one_week_end = end_date - 5 * pd.tseries.offsets.BDay()
#one_week_end = one_week_end.strftime("%Y-%m-%d")

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Show available calendars
print(mcal.get_calendar_names())
early = nyse.schedule(start_date, end_date)

nstocks = 300

2021-12-31
2007-01-19
['ASX', 'BMF', 'CFE', 'CBOE_Futures', 'CBOE_Equity_Options', 'CBOE_Index_Options', 'CME_Equity', 'CBOT_Equity', 'CME_Agriculture', 'CBOT_Agriculture', 'COMEX_Agriculture', 'NYMEX_Agriculture', 'CME_Rate', 'CBOT_Rate', 'CME_InterestRate', 'CBOT_InterestRate', 'CME_Bond', 'CBOT_Bond', 'EUREX', 'HKEX', 'ICE', 'ICEUS', 'NYFE', 'JPX', 'LSE', 'NYSE', 'stock', 'NASDAQ', 'BATS', 'DJIA', 'DOW', 'OSE', 'SIX', 'SSE', 'TSX', 'TSXV', 'BSE', 'TASE', 'AIXK', 'ASEX', 'BVMF', 'CMES', 'IEPA', 'XAMS', 'XASX', 'XBKK', 'XBOG', 'XBOM', 'XBRU', 'XBSE', 'XBUD', 'XBUE', 'XCBF', 'XCSE', 'XDUB', 'XFRA', 'XETR', 'XHEL', 'XHKG', 'XICE', 'XIDX', 'XIST', 'XJSE', 'XKAR', 'XKLS', 'XKRX', 'XLIM', 'XLIS', 'XLON', 'XMAD', 'XMEX', 'XMIL', 'XMOS', 'XNYS', 'XNZE', 'XOSL', 'XPAR', 'XPHS', 'XPRA', 'XSES', 'XSGO', 'XSHG', 'XSTO', 'XSWX', 'XTAE', 'XTAI', 'XTKS', 'XTSE', 'XWAR', 'XWBO', 'us_futures', '24/7', '24/5']


In [4]:
url = "ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqtraded.txt"

urllib.request.urlretrieve(url, "nasdaqtraded.txt")
urllib.request.urlretrieve(url, "mfundslist.txt")
urllib.request.urlretrieve(url, "bonds.txt")

df1 = pd.read_csv("nasdaqtraded.txt", sep="|")[0:-1]
df2 = pd.read_csv("mfundslist.txt", sep="|")[0:-1]
df3 = pd.read_csv("bonds.txt", sep="|")[0:-1]

# combined = pd.concat([df1['Symbol'],df2['Symbol'],df3['Symbol']],axis=0)

# process symbols for bad characters
BAD_CHARS = ["$", "."]
# pat = '|'.join(['({})'.format(re.escape(c)) for c in BAD_CHARS])
# cleaned = unique(combined.replace(BAD_CHARS,'-'))

# choose size
size = nstocks
# stocks = list(df1["Symbol"].sample(n=int(size/3)))
stocks = list(
    df1["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
mfunds = list(
    df2["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
bonds = list(
    df3["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
symbols = list(set(stocks + mfunds + bonds))  # unique(stocks + mfunds + bonds)
# symbols = unique(stocks)

In [5]:
def dl_data (npa_):
    name = npa_[0]
    s_ = npa_[1]
    e_ = npa_[2]    
    
    subset = pd.DataFrame()

    subset = yf.download(name, start=s_.strftime('%Y-%m-%d'), end=e_.strftime('%Y-%m-%d'), auto_adjust=True,progress=False,show_errors=False, threads = False).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[s_.strftime('%Y-%m-%d'):e_.strftime('%Y-%m-%d')]

    if len(subset) > 2:
        return (subset)
    else:
        return(0)
    

In [6]:
client = Client('192.168.3.100:8786',timeout=3)

client.ncores

npa = []

for s in range(0,len(symbols)):
    npa.append([symbols[s],start_date,one_week_end])

#results = await client.gather(futures, asynchronous=True)
    
future = client.map(dl_data,npa)

results = []
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        results.append("error")
    else:
        results.append(f.result())
    #print(f.result())

client.close()

vetted_symbols = []

for r in range(0,len(results)):
    if(isinstance(results[r], pd.DataFrame)):
        vetted_symbols.append(symbols[r])

In [7]:
len(vetted_symbols)

268

In [8]:
npa = []

for s in range(0,len(vetted_symbols)):
    npa.append([vetted_symbols[s],start_date,end_date])
    
client = Client('192.168.3.100:8786',timeout=3)

future = client.map(dl_data,npa)

symbols_completed = []
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        symbols_completed.append("error")
    else:
        symbols_completed.append(f.result())
    #print(f.result())

client.close()



In [9]:
symbols_completed_ = pd.DataFrame()

#commodities_includes = []
#names = []

#completed_2 = []

for x in range(0,len(symbols_completed)):
    
    if isinstance(symbols_completed[x], pd.DataFrame):
        #print(commodities[x])
        #print("Not")
        #commodities_includes.append(True)
        values = symbols_completed[x]
        values.index = pd.to_datetime(values.index)
        values = values.resample(frequency).mean().dropna()
        values['Symbol'] = symbols[x]
        values = values.loc[~values.index.duplicated(keep='last')]
        values = values.reset_index()
        #names.append(completed[x])

        #if(len(values))
        symbols_completed_ = pd.concat([symbols_completed_,values], axis=0)        
        
    #else:
        #print("None")
        

In [10]:
len(symbols_completed_)

7794

In [11]:
symbols_pvt = pd.pivot_table(symbols_completed_, columns=['Symbol'], index = 'Date', values=['Close'])

In [12]:
symbols_pvt

Close                                               \
Symbol           ACBA       ACMR        ACP        ACTV      AEHAW   
Date                                                                 
2007-03-31  17.168200        NaN        NaN   25.753909        NaN   
2007-06-30  21.067460        NaN        NaN   26.411872        NaN   
2007-09-30  24.114762        NaN        NaN   21.803448        NaN   
2007-12-31  28.031719        NaN        NaN   20.430607        NaN   
2008-03-31  23.621639        NaN        NaN   17.609631        NaN   
2008-06-30  27.021406        NaN        NaN   16.735757        NaN   
2008-09-30  26.953438        NaN        NaN   14.482854        NaN   
2008-12-31  16.047344        NaN        NaN    8.473682        NaN   
2009-03-31  14.436394        NaN        NaN    8.103052        NaN   
2009-06-30  19.252540        NaN        NaN   10.387963        NaN   
2009-09-30  24.893906        NaN        NaN   15.333246        NaN   
2009-12-31  27.296250        NaN        NaN   23.316817        NaN   
2010-03-31  34.037213        NaN        NaN   22.689115        NaN   
2010-06-30  34.441905        NaN   5.771027   23.519084        NaN   
2010-09-30  27.122344        NaN   5.950748   31.568012        NaN   
2010-12-31  29.933125        NaN   8.421930   40.648011        NaN   
2011-03-31  29.968226        NaN  10.161865   35.918608        NaN   
2011-06-30  34.681746        NaN  12.009683   30.817181        NaN   
2011-09-30  27.507031        NaN  10.868706   25.853976        NaN   
2011-12-31  26.307619        NaN  11.568143   18.078690        NaN   
2012-03-31  35.326774        NaN  14.778620   19.515615        NaN   
2012-06-30  30.568889        NaN  15.856786   12.659899        NaN   
2012-09-30  29.889524        NaN  19.050572   11.426698        NaN   
2012-12-31  25.509032        NaN  20.534229   13.283815        NaN   
2013-03-31  25.181833        NaN  22.846865   14.599299        NaN   
2013-06-30  27.778437        NaN  26.280830   13.234889        NaN   
2013-09-30  34.023906        NaN  25.200764   15.749336        NaN   
2013-12-31  39.625469        NaN  26.053810   23.583247        NaN   
2014-03-31  40.837049        NaN  28.362811   19.630289        NaN   
2014-06-30  41.803651        NaN  30.102920   28.113699        NaN   
2014-09-30  40.498750  26.159937  28.135488   35.538814        NaN   
2014-12-31  39.004531  25.433908  29.881706   35.913107        NaN   
2015-03-31  40.056393  27.160643  35.037638   44.968149        NaN   
2015-06-30  44.333809  27.869385  37.234554   48.127151        NaN   
2015-09-30  38.369844  26.520465  35.841280   40.589144        NaN   
2015-12-31  37.323281  28.014627  41.049909   49.412302        NaN   
2016-03-31  35.429836  25.691110  41.778081   57.893207        NaN   
2016-06-30  42.139063  20.132322  47.920032   59.888980        NaN   
2016-09-30  45.827187  22.613125  44.675078   69.652481        NaN   
2016-12-31  49.468889  29.652540  47.094017   66.221901  52.810988   
2017-03-31  55.747097  31.886855  51.001280   73.042988  56.174029   
2017-06-30  52.997937  34.336190  52.061571   81.111233  60.513195   
2017-09-30  52.629365  36.155317  48.749538   89.966130  63.305591   
2017-12-31  60.610794  37.370953  56.320639  108.584197  65.435029   
2018-03-31  61.389016  34.214263  58.125928  100.581676  67.637190   
2018-06-30  59.666094  31.560313  58.823868   80.678565  66.216760   
2018-09-30  61.183016  36.525079  61.864457   51.249223  64.917645   
2018-12-31  56.317460  32.150000  55.126040   40.180214  60.041656   
2019-03-31  57.738852  32.403115  52.940757   49.079517  60.804923   
2019-06-30  57.700952  37.506984  48.939189   50.728417  63.485082   
2019-09-30  59.295312  40.170156  52.220394   46.655000  64.077615   
2019-12-31  72.552813  43.084375  45.004016   26.938906  67.545696   
2020-03-31  67.466936  39.719436  29.505292   28.640968  64.530192   
2020-06-30  62.072063  30.722619  19.885556   18.501587  59.373678   
2020-09-30  73.331875  36.

In [13]:
# Show available calendars
# print(mcal.get_calendar_names())
'''
vetted_symbols = list(
    pf_pre.data.loc[
        (np.intersect1d(list(pf_pre.data.index.strftime("%Y-%m-%d")), early.index.strftime('%Y-%m-%d')))
    ]
    .head(-1)
    .tail(-1)
    .dropna(axis=1)
    .columns
)
'''

'\nvetted_symbols = list(\n    pf_pre.data.loc[\n        (np.intersect1d(list(pf_pre.data.index.strftime("%Y-%m-%d")), early.index.strftime(\'%Y-%m-%d\')))\n    ]\n    .head(-1)\n    .tail(-1)\n    .dropna(axis=1)\n    .columns\n)\n'

In [14]:

#Ultra-Low-Sulfur No. 2 Diesel Fuel Prices: Los Angeles (WDFUELLA)
#US Regular All Formulations Gas Price (GASREGW)
#Trade Weighted U.S. Dollar Index: Broad, Goods and Services (DTWEXBGS)


etf_commodities = ['DBO','CORN', 'WEAT', 'SOYB', 'JO', 'SGG', 'BAL', 'COW', 'MOO', 'TAGS', 'KOL' ]
#Gold, Silver, Platinum, Copper, Paladium, Aluminum, Iron, Steel
etf_metals = ['IAU', 'SLV', 'PGM', 'JJC', 'PALL', 'JJU', 'IFUNX', 'SLX']
#US dollar, European Euro, Japanese yen, Pound sterling, Australian dollar, Canadian dollar, Swiss franc, Chinese Yuan Renminbi, Swedish Krona, Peso, India
#defunct: Russia: XRU, Mexico: FXM
etf_foreign_exchanges = ['UUP','FXE','FXY','FXB','FXA','FXC','FXF','CYB', 'FXS', 'INR']
#residential, Ishares all NAmerica
etf_real_estate = ['REZ', 'IYR']
#Russia, Germany, UK, Japan, China, Euro, Euro, Brazil, Latin America, Mexico, India
etf_economies = ['ERUS','EWG','EWU','EWJ','MCHI','EZU','IEUR','EWZ','ILF','EWW','INDA']
#Ishares Investment Grade, IShares core aggregate Investment grade, Short, Total, 1-5 Years, 5-10 Years, 10 Years, Gov/Credit
#defunct:

etf_spdr_indexes = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
etf_dow_components = ['MMM','AXP','AMGN','AAPL','BA','CAT','CVX','CSCO','KO','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PG','CRM','TRV','UNH','VZ','V','WMT','WBA','DIS']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

crypto = ['BTC-USD','ETH','RPL','BCH','EOS','LTC']

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DTB3",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1252881600Q",\
"LEU0252918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MZMSL",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WTB3MS"    
]
#FRED_Indicators = ["CPALTT01USQ657N","PAYEMS","IRLTLT01USM156N","MABMM301USM189S","LFWA64TTUSM647S","MANMM101USA189S","MICH","UMCSENT","CSCICP03USM665S","DGS10","DTB3","DGS3MO","CASTHPI","GDPC1","CIVPART","POPTOTUSA647NWDB","MEHOINUSA672N","HOSMEDUSM052N","MORTGAGE30US","TTLHH","CSUSHPINSA","EMRATIO","CPIAUCSL","PSAVERT","LRUN64TTUSQ156S","USSTHPI","NYSTHPI","M2V","GFDEBTN","DFII10","GFDEGDQ188S","CUSR0000SEHA","ETOTALUSQ176N","ERENTUSQ176N","RECPROUSM156N","T5YIFR","BAMLHYH0A0HYM2TRIV","BAMLCC0A1AAATRIV","GVZCLS","DGS1","BAMLCC0A4BBBTRIV","VXVCLS","IC4WSA","WILLMICROCAPPR","WILLLRGCAPVAL","CFNAIDIFF","MZMSL","KCFSI","T5YIE","TOTALSA","USSLIND","AWHAETP","CES0500000003","TCU","WTB3MS","WGS3MO","TWEXB","DEXCHUS","DEXUSUK","CILACBQ158SBOG","CES4348400001","FEDFUNDS","TDSP","PERMIT","GFDEGDQ188S","CP","PRFI","DRSFRMACBS","DRCCLACBS","DRBLACBS","DALLCIACBEP","USROA","USROE","RSAHORUSQ156S","MEFAINUSA672N","COMREPUSQ159N","HDTGPDUSQ163N","POP","NROU","FGCCSAQ027S","TEDRATE", "VIXCLS", "NFCI","INDPRO","LES1252881600Q","CUUR0000SEHA","LEU0252918500Q","BAA10Y","BAMLC0A0CM","BAMLH0A3HYC","BOGMBASE","DCOILBRENTEU","DCOILWTICO","DFF","DGS1MO","DGS30","DGS5","FPCPITOTLZGUSA","GOLDAMGBD228NLBM","ICSA","INTDSRUSM193N","M1","M1V","MPRIME","PPIACO","SPCS20RSA","STLFSI2","T10Y2Y","T10Y3M","TB3MS","TREAST","UNRATE","WPU0911"]

Indexes = ['^SP500TR', '^GSPC', 'QQQ', 'DIA', 'VTWO', 'RUA','W5000FLT']

ManualStocks = ['VOO','SPY']
ManualStocks.extend(vetted_symbols)

etf_indexes_and_Crypto_list = [Indexes, ManualStocks, etf_commodities, etf_metals, etf_foreign_exchanges, etf_real_estate, etf_economies, etf_bonds, etf_muni_bonds, etf_treasuries, crypto, etf_spdr_indexes, etf_dow_components]

commodities = []
for sublist in etf_indexes_and_Crypto_list:
    for val in sublist:
        commodities.append(val)

#pool2 = concurrent.futures.ProcessPoolExecutor(cores)

completed = []
def dl(name):
    subset = yf.download(name, start=start_date, end=end_date, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[start_date.strftime('%Y-%m-%d'):end_date.strftime('%Y-%m-%d')]
    #sleep(1)
    if len(subset) != 0:
        completed.append(name)
        return (subset)
    
def dl2(assets):
    #https://webcache.googleusercontent.com/search?q=cache:Em9Ge5B9ue8J:https://stackoverflow.com/questions/62614282/is-the-yfinance-module-broken-i-keep-getting-a-jsondecodeerror+&cd=3&hl=en&ct=clnk&gl=us

    yahoo_financials = YahooFinancials(assets)

    #sleep(1)
    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [15]:
npa = []

for s in range(0,len(commodities)):
    npa.append([commodities[s],start_date,one_week_end])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(dl_data,npa)

completed_commodities = []
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        completed_commodities.append("error")
    else:
        completed_commodities.append(f.result())
    #print(f.result())

client.close()

vetted_commodities = []

for r in range(0,len(completed_commodities)):
    if(isinstance(completed_commodities[r], pd.DataFrame)):
        vetted_commodities.append(commodities[r])

#change end date
npa = []

for s in range(0,len(commodities)):
    npa.append([commodities[s],start_date,end_date])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(dl_data,npa)

commodities_data =[]
#best = -1
for f in as_completed(future):
    if(f.status=="error"):
        commodities_data.append("error")
    else:
        commodities_data.append(f.result())
    #print(f.result())    

client.close()
            

In [16]:
len(vetted_commodities)

356

In [17]:
commodities_ = pd.DataFrame()

#commodities_includes = []
#names = []

#completed_2 = []

for x in range(0,len(completed_commodities)):
    
    if isinstance(completed_commodities[x], pd.DataFrame):
        #print(commodities[x])
        #print("Not")
        #commodities_includes.append(True)
        values = completed_commodities[x]
        values.index = pd.to_datetime(values.index)
        values = values.resample(frequency).mean().dropna()
        values['Symbol'] = commodities[x]
        values = values.loc[~values.index.duplicated(keep='last')]
        values = values.reset_index()
        #names.append(completed[x])

        commodities_ = pd.concat([commodities_,values], axis=0)        
        
    #else:
        #print("None")
        

In [18]:
commodities_pvt = pd.pivot_table(commodities_, columns=['Symbol'], index = 'Date', values=['Close'])

In [19]:
commodities_pvt.describe()

Close                                                        \
Symbol       AAPL        ABEV       ACBA     ACMR      ACTV        ADAL   
count   60.000000   60.000000   3.000000  1.00000  4.000000   60.000000   
mean    31.501216  108.813601  27.489940  9.67875  9.727439   90.893642   
std     12.261709   14.196904   1.506423      NaN  0.028616   16.652937   
min     12.785431   77.275048  25.792680  9.67875  9.687625   58.492603   
25%     20.511962  100.175394  26.900707  9.67875  9.716192   78.305860   
50%     32.079290  109.716883  28.008734  9.67875  9.736107   92.577078   
75%     39.671503  116.063835  28.338571  9.67875  9.747354   99.852935   
max     55.117778  132.681187  28.668407  9.67875  9.749915  121.112826   

                                                                             \
Symbol       AGEN        AGG         AGL        AGRX         AGZ       AILV   
count   18.000000  60.000000   60.000000   33.000000   15.000000   4.000000   
mean    37.866505  17.293013  105.263105   72.759415   33.089454  12.447215   
std     21.320378   6.181838   57.695881   27.911063   26.700081   5.922654   
min      9.858279   5.224516   18.130065   38.960349   15.884284   5.259830   
25%     17.818853  15.763423   69.121099   48.661932   17.036649   9.418239   
50%     44.066680  17.793032  100.400355   71.598899   18.859088  12.614251   
75%     51.713220  20.444826  137.091972   90.266817   43.263046  15.643227   
max     72.702131  29.381309  251.838655  143.084237  110.879545  19.300526   

                                                                         \
Symbol       AIZN       AMGN      AMRN       ANGN       ANTM    APTV-PA   
count   60.000000  60.000000  3.000000  33.000000  22.000000  60.000000   
mean     6.688392  76.063704  9.812727  22.360755  62.468415  25.968009   
std      2.673035  10.228773  0.072084   3.175826   8.813096  10.096413   
min      1.050656  54.241150  9.760656  16.082307  49.422137  11.939599   
25%      4.817222  70.071896  9.771591  19.915028  57.116761  17.434594   
50%      7.009524  74.213022  9.782526  22.398767  61.135375  25.786870   
75%      8.274609  84.379762  9.838763  24.627582  63.908164  30.882371   
max     11.246984  95.774104  9.895000  27.829444  80.281250  56.363559   

                                                                          \
Symbol       AQWA       ASYS    ATCO-PH       ATRI       AVMU        AXP   
count   60.000000  60.000000   2.000000   9.000000  60.000000  60.000000   
mean    39.071090   8.349859  19.782378  23.355454  21.834838  19.048195   
std     25.400319   2.133451   0.534652   2.741962  18.392694   9.495689   
min     11.703983   2.701783  19.404322  18.462118   3.433242   6.241587   
25%     17.341859   6.804599  19.593350  21.274430   9.040084  10.032787   
50%     30.675100   8.619175  19.782378  24.130843  17.189234  21.480476   
75%     49.462300   9.944523  19.971407  25.697327  26.519452  26.044636   
max     95.680575  11.793373  20.160435  26.168828  88.883763  44.323965   

                                                                             \
Symbol         AXS          BA        BAB        BAL        BBP        BCAT   
count    60.000000   60.000000  60.000000   3.000000  42.000000   60.000000   
mean     34.312754   91.145423  44.715887  16.438160  19.890546  112.778274   
std      24.002226   14.549334  19.727151   0.629088   3.381105   52.708199   
min       8.103052   64.874913  14.641401  15.721371  13.745079   35.446758   
25%      15.797295   81.107388  23.597871  16.207943  17.571749   61.079536   
50%      25.803942   91.685318  46.045689  16.694516  19.336275  114.669481   
75%      47.023038   98.928649  63.610146  16.796554  22.878250  156.421895   
max     108.584197  116.624778  76.962188  16.898593  26.930159  209.287650   

                                                                           \
Symbol        BCH       BCOV      BEP-PA       BGRN       BGX       BHFAP   
count   40.00

In [20]:
def Fred_Data(npa_):
    name = npa_[0]
    s_ = npa_[1]
    e_ = npa_[2]    
    
    '''
    subset = pd.DataFrame()

    subset = yf.download(name, start=s_.strftime('%Y-%m-%d'), end=e_.strftime('%Y-%m-%d'), auto_adjust=True,progress=False,show_errors=False, threads = False).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[s_.strftime('%Y-%m-%d'):e_.strftime('%Y-%m-%d')]

    if len(subset) != 0:
        return (subset)
    else:
        return(0)
    '''
    temp = web.get_data_fred(str(name), s_.strftime('%Y-%m-%d'), e_.strftime('%Y-%m-%d'))
    temp.index = pd.to_datetime(temp.index)
    temp = temp.resample(frequency).mean().dropna()
    return(temp)

npa = []

for s in range(0,len(FRED_Indicators)):
    npa.append([FRED_Indicators[s],start_date,one_week_end])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(Fred_Data,npa)

FRED_completed = []

for f in as_completed(future):
    if(f.status=="error"):
        FRED_completed.append("error")
    else:
        FRED_completed.append(f.result())
    #print(f.result())

client.close()



npa = []

for s in range(0,len(FRED_completed)):
    npa.append([FRED_completed[s],start_date,one_week_end])

client = Client('192.168.3.100:8786',timeout=3)

future = client.map(Fred_Data,npa)

FRED_set = []

for f in as_completed(future):
    if(f.status=="error"):
        FRED_completed.append("error")
    else:
        FRED_completed.append(f.result())
    #print(f.result())

client.close()


In [21]:
FRED_set_ = pd.DataFrame()
#FRED_set_ = []

#commodities_includes = []
#names = []

#completed_2 = []

for x in range(0,len(FRED_completed)):
    
    if isinstance(FRED_completed[x], pd.DataFrame):
        #print(commodities[x])
        #print("Not")
        #commodities_includes.append(True)
        
        values = FRED_completed[x]
        values.index = pd.to_datetime(values.index)
        values = values.resample(frequency).mean().dropna()
        
        values = values.loc[~values.index.duplicated(keep='last')]
        values = values.reset_index()
        values['Symbol'] = values.columns[1]
        values.columns = ['DATE','Value','Symbol']
        #names.append(completed[x])

        #FRED_set_.append(values)#
        FRED_set_ = pd.concat([FRED_set_,values], axis=0)        
        
    #else:
        #print("None")
        

In [22]:
FRED_set_.columns

Index(['DATE', 'Value', 'Symbol'], dtype='object')

In [23]:
FRED_set_pvt = pd.pivot_table(FRED_set_, columns=['Symbol'], index = 'DATE', values='Value')

In [24]:
FRED_set_pvt

Symbol,ASPUS,AWHAETP,B432RG3A086NBEA,BAA10Y,BACDINA066MNFRBNY,BACTSAMFRBDAL,BAMLC0A0CM,BAMLCC0A1AAATRIV,BAMLCC0A4BBBTRIV,BAMLH0A3HYC,BAMLHYH0A0HYM2TRIV,BOGMBASE,BOGZ1FA105015103Q,BOGZ1FA145050005Q,BOGZ1FA205050005Q,BOGZ1FA315050005Q,BOGZ1FA385050005Q,BOGZ1FL105015105Q,BUSAPPWNSACA,CASTHPI,CES0500000003,CES4348400001,CFNAI,CFNAIDIFF,CFSBCACTIVITYMFG,CILACBQ158SBOG,CIVPART,COMREPUSQ159N,CP,CPALTT01USQ657N,CPIAUCSL,CSCICP03USM665S,CSUSHPINSA,CUSR0000SEHA,CUUR0000SEHA,DALLCIACBEP,DCOILBRENTEU,DCOILWTICO,DEXCHUS,DEXUSUK,DFF,DFII10,DGORDER,DGS1,DGS10,DGS1MO,DGS2,DGS30,DGS3MO,DGS5,DRBLACBS,DRCCLACBS,DRSFRMACBS,DTB3,DTWEXBGS,EMRATIO,ERENTUSQ176N,ETOTALUSQ176N,FEDFUNDS,FGCCSAQ027S,FPCPITOTLZGUSA,GASREGW,GDPC1,GFDEBTN,GFDEGDQ188S,GOLDAMGBD228NLBM,GVZCLS,HDTGPDUSQ163N,HOSMEDUSM052N,IC4WSA,ICSA,IIPUSNETIQ,INDPRO,INTDSRUSM193N,IRLTLT01USM156N,KCFSI,LES1252881600Q,LEU0252918500Q,LFWA64TTUSM647S,LRUN64TTUSQ156S,LXXRCSA,M1,M1V,M2V,MABMM301USM189S,MANMM101USA189S,MEFAINUSA672N,MEHOINUSA672N,MEPAINUSA672N,MICH,MORTGAGE30US,MPRIME,MSPUS,MZMSL,NFCI,NROU,NYSTHPI,PAYEMS,PERMIT,POP,POPTOTUSA647NWDB,PPIACO,PRFI,PSAVERT,RECPROUSM156N,RSAHORUSQ156S,SPCS20RSA,STLFSI2,T10Y2Y,T10Y3M,T10YIE,T5YIE,T5YIFR,TB3MS,TCU,TDSP,TEDRATE,TOTALSA,TREAST,TTLHH,TWEXB,U6RATE,UMCSENT,UNRATE,USPHCI,USREC,USROA,USROE,USSLIND,USSTHPI,VIXCLS,VXVCLS,WDFUELLA,WGS3MO,WILLLRGCAPVAL,WILLMICROCAPPR,WPU0911,WTB3MS
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-03-31,NaN,34.350000,NaN,1.621200,14.800000,14.950000,0.895192,402.519423,418.626923,4.710192,590.453654,8.204500e+05,NaN,NaN,NaN,NaN,NaN,NaN,6595.454545,NaN,20.700000,1451.950000,0.210000,0.065000,NaN,NaN,66.250000,NaN,NaN,NaN,204.757000,100.521300,182.332500,231.947500,232.117000,NaN,59.024400,58.900800,7.748572,1.955616,5.258889,2.297400,225244.000000,5.003800,4.673400,5.155600,4.746800,4.797000,5.121200,4.630800,NaN,NaN,NaN,4.984000,97.359439,63.300000,NaN,NaN,5.260000,NaN,NaN,2.369200,NaN,NaN,NaN,656.903922,NaN,NaN,NaN,3.176591e+05,3.164545e+05,NaN,100.917650,6.250000,4.640000,-0.648642,NaN,NaN,1.950941e+08,NaN,278.518638,1366.140000,NaN,NaN,7.140050e+12,NaN,NaN,NaN,NaN,3.000000,6.217000,8.250000,NaN,7307.900000,-0.690563,NaN,NaN,137675.500000,1597.000000,300911.500000,NaN,168.050000,NaN,3.750000,0.250000,NaN,204.009420,-0.965636,-0.073400,-0.447800,2.376000,2.396000,2.356000,4.985000,80.551400,NaN,0.370400,16.803500,7.798614e+05,NaN,107.188140,8.050000,89.850000,4.450000,99.360000,0.000000,NaN,NaN,1.215000,NaN,12.842200,NaN,1.968727,5.121818,39203.248200,7941.823800,157.000000,4.985455
2007-06-30,310100.0,34.400000,NaN,1.644219,20.333333,12.700000,0.942969,403.545781,419.222969,4.468750,600.548594,8.253000e+05,442864.0,1758835.0,352904.0,245570.0,4200623.0,4950911.0,5830.000000,623.57,20.860000,1447.400000,-0.033333,0.050000,NaN,12.7,65.966667,12.905,1400.804,1.917329,206.631000,100.120858,181.852333,233.633000,233.533333,13854.0,68.594516,64.965238,7.678400,1.986228,5.252418,2.439062,230374.666667,4.934219,4.846094,4.776719,4.803750,4.988906,4.872031,4.760781,1.14,4.02,2.29,4.737500,94.926737,63.000000,3763.0,128888.0,5.250000,111854.0,NaN,3.002538,15577.779,8867677.0,61.57219,667.617213,NaN,98.220,NaN,3.122115e+05,3.133846e+05,-1421382.0,101.802300,6.250000,4.846667,-0.559987,335.0,1092.0,1.954455e+08,4.564817,275.470797,1375.876923,10.469,1.987,7.249600e+12,NaN,NaN,NaN,NaN,3.333333,6.358462,8.250000,242200.0,7483.833333,-0.621124,4.933269,640.69,137968.000000,1456.666667,301492.000000,NaN,172.833333,719.274,3.633333,0.393333,68.3,200.920714,-0.743138,0.042344,-0.025937,2.407031,2.408594,2.405469,4.736667,80.816000,12.941819,0.627705,16.485000,7.879361e+05,NaN,104.776800,8.233333,86.900000,4.500000,99.860000,0.000000,1.23,12.13,0.983333,380.66,13.731905,NaN,2.186077,4.870769,41440.487143,8151.327302,160.266667,4.736923
2007-09-30,301200.0,34.400000,NaN,1.895714,13.500000,0.200000,1.328485,408.220606,419.269545,6.413485,58